# Introduction

This is the record of my study progress on NTK. The plan is as follows.

- Study the Neural Tangent Library by Google
- Reproduce several works published in 2019 focusing on the basic property of NTK and being familiar with the code
- TBD

# Library

The following paper contains the development of the [neural-tangents](https://github.com/google/neural-tangents) library

- [Neural Tangents: Fast and Easy Infinite Neural Networks in Python](https://arxiv.org/abs/1912.02803)
- [Fast Finite Width Neural Tangent Kernel](https://arxiv.org/abs/2206.08720)
- [Infinite attention: NNGP and NTK for deep attention networks](https://arxiv.org/abs/2006.10540)
- [On the infinite width limit of neural networks with a standard parameterization](https://arxiv.org/abs/2001.07301)
- [Fast Neural Kernel Embeddings for General Activations](https://arxiv.org/abs/2209.04121)

# First Results

The following paper contains the basic property of NTK in the early years, which will be used to be familiar with the [neural-tangents](https://github.com/google/neural-tangents) library.

- NIPS 2019
  - [Wide Neural Networks of Any Depth Evolve as Linear Models Under Gradient Descent](https://arxiv.org/pdf/1902.06720.pdf)
  - [On Lazy Training in Differentiable Programming](https://arxiv.org/pdf/1812.07956.pdf)
  - [The Convergence Rate of Neural Networks for Learned Functions of Different Frequencies](https://arxiv.org/pdf/1906.00425.pdf)
  - [Limitations of Lazy Training of Two-layers Neural Networks](https://arxiv.org/pdf/1906.08899.pdf)
  - [On the Inductive Bias of Neural Tangent Kernels](https://arxiv.org/pdf/1905.12173.pdf)
- ICML 2020
  - [Neural Kernels Without Tangents](https://arxiv.org/pdf/2003.02237.pdf)
- arXiv
  - [A Fine-Grained Spectral Perspective on Neural Networks](https://arxiv.org/pdf/1907.10599.pdf)